**UNIVERSITY OF VICTORIA**\
**CSC 502 PROJECT (202001)**\
**SUBMITTED BY**\
Suseela Pattamatta


In [0]:
!rm -rf *.txt
!rm -rf *.db
!ls

sample_data


FUNCTION TO ESTABLISH DATABASE CONNECTION


In [0]:
def create_db_connection(db_connection):
    conn = None
    try:
        conn = sqlite3.connect(db_connection)
        print("connection to database is successful")
    except Error as e:
        print(e)
    return conn

FUNCTIONS TO CREATE POSTINGS AND DOCMAG TABLES

In [0]:
def create_postings(conn):
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS postings(word VARCHAR(100) PRIMARY KEY, postinglist_freq_tfidf TEXT)")

def create_docmag(conn):
    cur = conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS docmag(docid INT PRIMARY KEY, maxf INT, mag FLOAT)")

FUNCTIONS TO INSERT INTO POSTINGS AND DOCMAG TABLES

In [0]:
def insert_postings(conn,postings_text):

    cur = conn.cursor()
    cur.executemany("Insert into postings VALUES(?,?)",postings_text)

def insert_doc_mag(conn,docmag_data):
    cur = conn.cursor()
    cur.executemany("Insert into docmag VALUES(?,?,?)",docmag_data)

CREATE THE DATABASE\
CREATE POSTINGS AND DOCMAG TABLES\
INSERT DATA INTO POSTINGS AND DOCMAG TABLES

In [0]:
def create_database_and_insert_data():
  database = "/content/502Proj.db"
  conn = create_db_connection(database)
  if (conn != 'None'):
          create_postings(conn)
          create_docmag(conn)

  print("start inserting of postings")  
  f_postings =[x.strip('\n') for x in open("inv_idx.txt")]
  postings = [x[1:-1].replace("'","").split(',',1) for x in f_postings]
  insert_postings(conn,postings)
  print("inserting postings completed")


  print("start inserting into docmag")  
  f = [x.strip('\n') for x in open("doc_mag.txt")]
  f_docmag_data = [re.findall('[0-9]+[.]?[0-9]*', x) for x in f]  
  temp = []
  for item in f_docmag_data:    
    if len(item) > 3:
      item[2] = item[2]+"e-"+item[3]
      item = item[:-1]       

    temp.append([int(item[0]), int(item[1]), float("%.10f" % float(float(item[2])) )])
        
  docmag_data = temp  
  insert_doc_mag(conn,docmag_data)
  print("finish inserting into docmag")
  
  return conn
  

KEYWORD SEARCH FUNCTIONALITY


In [0]:
def word_search(postings2) :
  for i in postings2.postinglist_freq_tfidf:
      i = i.replace("[","").replace("]","").replace("(","").strip().split("),")
  document_list=[]
  for j in i:
    j = j.split(",")
    
    document_list.append((j[0].strip(),j[1].strip(),j[2].strip().replace(")","")))

  return (document_list)

FUNCTION TO GET ALL DOCUMENT MAGNITUDES FROM DATABASE



In [0]:
def get_doc_mags(conn):
    query = "select * from docmag " 
    docMags = pd.read_sql_query(query,conn)
    pd.set_option('max_colwidth', 800)
    # print(docMags)

    docmags_dict={}
    for i, row in docMags.iterrows():
      docmags_dict[str(int(row['docid']))]=row['mag']
    return docmags_dict

CALCULATING IDF AND MAGNITUDE FOR QUERY AND DOT-PRODUCT VALUES (numerators of cosine similarity)

In [0]:
def query_idf(conn, input_query):
  
  terms = re.findall('[a-zA-Z]+', input_query)
  terms = [term.lower() for term in terms]
  terms = [term for term in terms if term not in stop_words]
    
  maxf = 0
  dict_terms = {}
  for term in terms:
      if term not in dict_terms:
        dict_terms[term] = 1
      else:
        dict_terms[term] += 1

      if dict_terms[term] > maxf:        
        maxf = dict_terms[term]
  
  final_list = []
  for key, value in dict_terms.items():
      final_list.append((key, (value/maxf)))

  dict_idf = {}
  final_list1 = []
  term_doc_list = {}

  for term in terms:

      query = "select * from postings where word = '%s'" %term      
      postings2 = pd.read_sql_query(query,conn)
      pd.set_option('max_colwidth', 800)
      
      if list(postings2.postinglist_freq_tfidf) != []:
        term_doc_list[term] = word_search(postings2)
        dict_idf[term] = float(1/len(term_doc_list[term]))
      else:
        if term in dict_terms:
          del dict_terms[term]

  # Calculate query magnitude
  query_magnitude = 0
  for key, value in dict_terms.items():
      value = (value/maxf) * dict_idf[key]
      final_list1.append((key, value,term_doc_list[key]))
      query_magnitude = query_magnitude + (value**2)
  # print("query magnitude is : ",query_magnitude) 
 
  idf = pd.DataFrame(final_list1,columns = ['word','query_tfidf','doc_tfidf'])
  
  # Calculate dot-product values
  dict_final = {}

  for i in final_list1:    

    term_list = []
    key = i[0]
    multi = i[1]
    for term in i[2]:
        term_list.append((term[0],float(term[2])*multi))
    dict_final[key]=term_list
   
  value = 0
  dict_dp = {}
  doc = dict()
  list_dp=[]
  new_dict={}
  for key in sorted(dict_final):
    for term in dict_final[key]:     
                
      if term[0] in new_dict:
        new_dict[term[0]] = new_dict[term[0]]+term[1]
      else:
        new_dict[term[0]] = term[1]

  # print(new_dict)
  return new_dict, query_magnitude

CALCULATE COSINE SIMILARITY AND RANK THE DOCUMENTS BASED ON COSINE SIMILARITY

In [0]:
def cosine_sim(docmags_dict, new_dict, query_magnitude):
  cos_q_doc={}
  for key in docmags_dict:
    if key in new_dict:
        cos_q_doc[key] = new_dict[key]/(query_magnitude*docmags_dict[key])
          
    else:
      cos_q_doc[key] = 0
  
  # Sort based in cosine similarity
  import operator
  sorted_dict = sorted(cos_q_doc.items(), key=operator.itemgetter(1),reverse=True)

  print("\nWe have now processed the cosine similarities between the query and the documents. ")
  number_lines = int(input("How many top similar documents do you want to see?"))
  print("\nThe descending order of the document with respect to cosine similarity to the query is : ","\n")
  count = 0 
  for key,value in sorted_dict:    
    print("The docid is : ",int(key),"      Cosine similarity is : ",value)
    
    count += 1
    if count > number_lines-1:
      break

USER INTERFACE\
**Note for Marker: ** When running this cell, you will be prompted to upload the text files with idf and magnitudes. Select inv_idx.txt and doc_mag.txt.
Then you will be asked to enter your query. 


In [0]:
import sqlite3
from sqlite3 import Error
import re
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import copy

from google.colab import files
# uploaded = files.upload()

def main():

  !ls
  !rm -rf 502Proj.db
  !ls 

  # Create the database and insert data in tables
  conn = create_database_and_insert_data()

  option = input("You may choose from 2 options\n * To use word search functionality, enter --> 1\n * To enter a query and get a ranking based on cosine similarity, enter --> 2\n")

  if option == "1":

    #Word search functionality
    print("This is the word search functionality")
    print(("Enter a word/term you want to search"))
    input_word = input()
    input_word = input_word.lower()
    query = "select * from postings where word = '%s'" %input_word
    postings2 = pd.read_sql_query(query,conn)
    if postings2.empty:
      print("The word/term :",input_word, " is not available in any of the documents")
    else:
      word_list = word_search(postings2)
      print("The word/term ",input_word," is available in the following documents : ")
      for k in word_list :
        print("\ndocid : ",k[0]," frequency : ",k[1]," tfidf: ",k[2])

  else:
    query = input("Enter your query: ")
    query = query.lower() 

    # Determine query idf
    new_dict, query_magnitude = query_idf(conn, query)
    
    # Get Document Magnitudes
    doc_mags = get_doc_mags(conn)
    
    # Calculate cosine similarities
    cosine_sim(doc_mags, new_dict, query_magnitude)

main()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
502Proj.db  doc_mag.txt  inv_idx.txt  sample_data
doc_mag.txt  inv_idx.txt  sample_data
connection to database is successful
start inserting of postings
inserting postings completed
start inserting into docmag
finish inserting into docmag
You may choose from 2 options
 * To use word search functionality, enter --> 1
 * To enter a query and get a ranking based on cosine similarity, enter --> 2
2
Enter your query: FEB and february

We have now processed the cosine similarities between the query and the documents. 
How many top similar documents do you want to see?10

The descending order of the document with respect to cosine similarity to the query is :  

The docid is :  1       Cosine similarity is :  0.04620545453615401
The docid is :  5       Cosine similarity is :  0.02711714607105407
The docid is :  4       Cosine similarity is :  0.024249784325681013
The docid is